In [ ]:
# Instalação da library
!pip install google-generativeai

# Ask to Code

## Introdução

Este projeto demonstra a criação de um chatbot que funciona como documentação de um repositório no Github, utilizando a API Gemini.



# Bibliotecas utilizadas

- genai: Gemini API
- userdata: Acessar variáveis de ambiente (secrets) no Google Colab
- requests: Acessar endpoints
- time: Biblioteca de temporização de código
- re: Detecção de padrões em linguagem regex

In [9]:
import google.generativeai as genai
from google.colab import userdata
import requests
import time
import re

# Configurações do Modelo

Adicione a Gemini API Key às variáveis secretas do Google Colab ou substitua "userdata.get('GEMINI_API_KEY')" pela string completa.



In [10]:
genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

generation_config = {
    "temperature": 0.9,
    "top_p": 1,
    "top_k": 0,
    "max_output_tokens": 8192,
}

safety_settings = [
    {
        "category": "HARM_CATEGORY_HARASSMENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_HATE_SPEECH",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_MEDIUM_AND_ABOVE"
    },
]

model = genai.GenerativeModel(
    model_name="gemini-1.5-pro-latest",
    generation_config=generation_config,
    safety_settings=safety_settings
)

# Aquisição de dados do repositório

Definição dos métodos utilizados para adquirir informações do repositório:
- Nome de usuário do proprietário
- Nome do repositório
- Arquivos e diretórios

In [11]:
not_allowed_files = ['.jpg', '.jpeg', '.png', '.gif', '.mp4', '.avi', '.mov', '.pdf', '.tar.gz', '.zip', '.rar']

def get_repo_info(repo_url):
  pattern = r"https://github.com/([^/]+)/([^/]+)(.*)"
  match = re.match(pattern, repo_url)
  if match:
    username = match.group(1)
    repo_name = match.group(2)
    return repo_name, username
  else:
    raise Exception('Github Link without owner name and/or repository name, please enter the correct one, e.g. https://github.com/{owner}/{repository}')

def get_content(url_repo):
    content_list = []

    def process_content(content):
        if content['type'] == 'file':
            file_name = content['name']
            if not any(file_name.endswith(filetype) for filetype in not_allowed_files):
                file_url = content['download_url']
                file_content = requests.get(file_url).text
                content_list.append({'role': 'user', 'parts': [file_content]})
                content_list.append({'role': 'user', 'parts': [file_url]})

        elif content['type'] == 'dir':
            folder_url = content['url']
            folder_response = requests.get(folder_url)
            if folder_response.status_code == 200:
                folder_contents = folder_response.json()
                for item in folder_contents:
                    process_content(item)

    response = requests.get(url_repo)
    if response.status_code == 200:
        contents = response.json()

        for content in contents:
            time.sleep(0.1)
            process_content(content)
    else:
        raise Exception("Error accessing the repository.")

    return content_list

# Utilização

## Repositório

Informe o link do github referente ao repositório a ser utilizado na criação do chatbot

## Prompts

Faça perguntas em linguagem natural em qualquer língua desejada para que o Gemini responda sobre informações com relação ao repositório informado

Envie 'end' para finalizar a conversa

In [17]:
repo_url = input("Enter the repository name: ")
repo_name, owner = get_repo_info(repo_url)

url_repository = f'https://api.github.com/repos/{owner}/{repo_name}/contents'
contents = get_content(url_repository)

conversation = model.start_chat(history=contents)

while True:
    prompt = input("Enter your message: ")

    if prompt.lower() == "end":
        break

    conversation.send_message(prompt)
    print(conversation.last.text)

print("Chat ended.")


Enter the repository name: https://github.com/pagarme/node-boleto
Enter your message: end
Chat ended.
